In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import warnings
warnings.filterwarnings('always')
warnings.filterwarnings('ignore')

# data visualisation and manipulation
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import style
import seaborn as sns
 
#configure
# sets matplotlib to inline and displays graphs below the corressponding cell.
%matplotlib inline  
style.use('fivethirtyeight')
sns.set(style='whitegrid',color_codes=True)

#model selection
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score,precision_score,recall_score,confusion_matrix,roc_curve,roc_auc_score
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import LabelEncoder

#preprocess.
from keras.preprocessing.image import ImageDataGenerator

#dl libraraies
from keras import backend as K
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import *
from tensorflow.keras.utils import to_categorical

# specifically for cnn
from keras.layers import Dropout, Flatten,Activation
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
 
import tensorflow as tf
import random as rn


In [ ]:
train_datagen = ImageDataGenerator(rescale=1 / 255.0,
                                   rotation_range=20,
                                   zoom_range=0.05,
                                   width_shift_range=0.05,
                                   height_shift_range=0.05,
                                   shear_range=0.05,
                                   horizontal_flip=True,
                                   fill_mode="nearest",
                                   validation_split=0.20)

test_datagen = ImageDataGenerator(rescale=1 / 255.0,
                                  validation_split=0.20)

data_dir='/content/drive/MyDrive/Deber/Granadilla_Aument'
img_height = 250
img_width = 250
batch_size = 32

train_generator = train_datagen.flow_from_directory(
    directory=data_dir,
    class_mode='categorical',
    color_mode='rgb',
    subset="training",
    seed=123,
    target_size=(img_height, img_width))

valid_generator = test_datagen.flow_from_directory(
    directory=data_dir,
    class_mode='categorical',
    color_mode='rgb',
    subset="validation",
    seed=123,
    target_size=(img_height, img_width),
    batch_size=batch_size)
test_generator = test_datagen.flow_from_directory(
    directory=data_dir,
    color_mode="rgb",
    batch_size=1,
    shuffle=False,
    seed=42
)

Found 4041 images belonging to 2 classes.
Found 1009 images belonging to 2 classes.
Found 5050 images belonging to 2 classes.


In [ ]:
import tensorflow as tf

model = tf.keras.Sequential([
    tf.keras.applications.MobileNetV2(include_top=False,
                                      input_shape=(100, 100, 3),
                                      weights="imagenet",
                                      pooling='avg',
                                      classes=1000),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(98, activation='relu'),
    tf.keras.layers.Dropout(0.25),
    tf.keras.layers.Dense(2, activation='softmax')
])

model.summary()

9420800/9406464 [==============================] - 0s 0us/step
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 mobilenetv2_1.00_224 (Funct  (None, 1280)             2257984   
 ional)                                                          
                                                                 
 dense (Dense)               (None, 128)               163968    
                                                                 
 dense_1 (Dense)             (None, 98)                12642     
                                                                 
 dropout (Dropout)           (None, 98)                0         
                                                                 
 dense_2 (Dense)             (None, 2)                 198       
                                                                 
Total params: 2,434,792
Trainable params: 2,400,680
Non-tra

In [ ]:
model.compile(optimizer='SGD',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Finally train this neural net. Ten epochs seems ok to me
history = model.fit_generator(train_generator, validation_data=valid_generator, epochs=5)

Epoch 1/5
127/127 [==============================] - 77s 583ms/step - loss: 0.0829 - accuracy: 0.9671 - val_loss: 0.2757 - val_accuracy: 0.8952
Epoch 2/5
127/127 [==============================] - 72s 566ms/step - loss: 0.0765 - accuracy: 0.9701 - val_loss: 0.3337 - val_accuracy: 0.8700
Epoch 3/5
127/127 [==============================] - 70s 553ms/step - loss: 0.0702 - accuracy: 0.9706 - val_loss: 0.5154 - val_accuracy: 0.8226
Epoch 4/5
127/127 [==============================] - 72s 564ms/step - loss: 0.0684 - accuracy: 0.9750 - val_loss: 0.5662 - val_accuracy: 0.8246
Epoch 5/5
127/127 [==============================] - 72s 567ms/step - loss: 0.0744 - accuracy: 0.9715 - val_loss: 0.3467 - val_accuracy: 0.8649


In [ ]:
STEP_SIZE_TEST=test_generator.n//valid_generator.batch_size
test_generator.reset()
pred=model.predict_generator(valid_generator,
steps=STEP_SIZE_TEST,
verbose=1)

 32/157 [=====>........................] - ETA: 10s

157/157 [==============================] - 4s 19ms/step


In [ ]:
STEP_SIZE_TRAIN=train_generator.n//train_generator.batch_size
STEP_SIZE_VALID=valid_generator.n//valid_generator.batch_size
model.fit_generator(generator=train_generator,
                    steps_per_epoch=STEP_SIZE_TRAIN,
                    validation_data=valid_generator,
                    validation_steps=STEP_SIZE_VALID,
                    epochs=10
)

Epoch 1/10
126/126 [==============================] - 69s 547ms/step - loss: 0.1815 - accuracy: 0.9349 - val_loss: 0.4565 - val_accuracy: 0.7681
Epoch 2/10
126/126 [==============================] - 71s 562ms/step - loss: 0.1631 - accuracy: 0.9369 - val_loss: 0.5559 - val_accuracy: 0.6643
Epoch 3/10
126/126 [==============================] - 70s 551ms/step - loss: 0.1579 - accuracy: 0.9396 - val_loss: 0.5169 - val_accuracy: 0.6915
Epoch 4/10
126/126 [==============================] - 69s 548ms/step - loss: 0.1390 - accuracy: 0.9476 - val_loss: 0.4192 - val_accuracy: 0.7984
Epoch 5/10
126/126 [==============================] - 68s 538ms/step - loss: 0.1403 - accuracy: 0.9424 - val_loss: 0.2882 - val_accuracy: 0.8861
Epoch 6/10
126/126 [==============================] - 68s 537ms/step - loss: 0.1278 - accuracy: 0.9549 - val_loss: 0.5476 - val_accuracy: 0.7792
Epoch 7/10
126/126 [==============================] - 69s 546ms/step - loss: 0.1142 - accuracy: 0.9549 - val_loss: 0.4060 - val_ac

In [ ]:
model.evaluate_generator(generator=valid_generator,
steps=STEP_SIZE_VALID)

[0.5368839502334595, 0.7651209831237793]